# 文件解析和文件生成

In [3]:
import numpy as np
from io import StringIO


## 网格文件

In [8]:
def parse_gridfile(filepath):
    f=open(filepath)
    data = []
    for line in f:
        data.append(line.strip())
    f.close()
    columns = int(data[0].split(" ",4)[0])
    rows = int(data[0].split(" ",4)[1])
    bend_penalty = int(data[0].split(" ",4)[2])
    via_penalty = int(data[0].split(" ",4)[3])
    print("bend_penalty:",bend_penalty,"\n","via_penalty:",via_penalty)
    string = str(data[1:])
    for item in ["'","[","]",","]:
        string=string.replace(item,"")
    array = np.genfromtxt(StringIO(string))
    array = array.reshape((2*rows,columns))
    layer1_gird,layer2_grid = np.vsplit(array, 2)[0],np.vsplit(array, 2)[1]
    print("size of layer1:",layer1_gird.shape)
    print("size of layer2:",layer2_grid.shape)
    return rows,columns,bend_penalty,via_penalty,layer1_gird,layer2_grid


In [9]:
rows,columns,bend_penalty,via_penalty,layer1_gird,layer2_grid = parse_gridfile('../benchmark/bench1.grid')


bend_penalty: 5 
 via_penalty: 1
size of layer1: (50, 50)
size of layer2: (50, 50)


## 网表文件

In [10]:
def parse_netlist(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    net_count = int(lines[0])
    nets = []

    for i in range(1, net_count + 1):
        net_data = lines[i].split()
        net_id = int(net_data[0])
        pin1_layer = int(net_data[1])
        pin1_x = int(net_data[2])
        pin1_y = int(net_data[3])
        pin2_layer = int(net_data[4])
        pin2_x = int(net_data[5])
        pin2_y = int(net_data[6])
        net = {
            'net_id': net_id,
            'pin1': {'x': pin1_x, 'y': pin1_y, 'layer': pin1_layer},
            'pin2': {'x': pin2_x, 'y': pin2_y, 'layer': pin2_layer}
        }
        nets.append(net)
    return nets



In [11]:
netlist_file_path = '../benchmark/bench1.nl'
nets = parse_netlist(netlist_file_path)
print(nets)

[{'net_id': 1, 'pin1': {'x': 3, 'y': 36, 'layer': 1}, 'pin2': {'x': 3, 'y': 29, 'layer': 1}}, {'net_id': 2, 'pin1': {'x': 4, 'y': 36, 'layer': 1}, 'pin2': {'x': 4, 'y': 29, 'layer': 1}}, {'net_id': 3, 'pin1': {'x': 5, 'y': 36, 'layer': 1}, 'pin2': {'x': 6, 'y': 29, 'layer': 1}}, {'net_id': 4, 'pin1': {'x': 7, 'y': 36, 'layer': 1}, 'pin2': {'x': 9, 'y': 29, 'layer': 1}}, {'net_id': 5, 'pin1': {'x': 10, 'y': 36, 'layer': 1}, 'pin2': {'x': 13, 'y': 29, 'layer': 1}}, {'net_id': 6, 'pin1': {'x': 49, 'y': 36, 'layer': 1}, 'pin2': {'x': 36, 'y': 36, 'layer': 1}}, {'net_id': 7, 'pin1': {'x': 49, 'y': 35, 'layer': 1}, 'pin2': {'x': 36, 'y': 35, 'layer': 1}}, {'net_id': 8, 'pin1': {'x': 49, 'y': 34, 'layer': 1}, 'pin2': {'x': 36, 'y': 33, 'layer': 1}}, {'net_id': 9, 'pin1': {'x': 49, 'y': 32, 'layer': 1}, 'pin2': {'x': 36, 'y': 30, 'layer': 1}}, {'net_id': 10, 'pin1': {'x': 49, 'y': 29, 'layer': 1}, 'pin2': {'x': 36, 'y': 26, 'layer': 1}}, {'net_id': 11, 'pin1': {'x': 12, 'y': 15, 'layer': 1}, '

## 输出文件生成

In [2]:
def generate_output_file(filepath, net_number, routing_paths):
    with open(filepath, 'w') as file:
        file.write(str(net_number) + '\n')

        for net_id, routing_path in routing_paths.items():
            file.write(str(net_id) + '\n')
            for layer, x, y in routing_path:
                file.write(str(layer) + ' ' + str(x) + ' ' + str(y) + '\n')
            file.write('0\n')

# Example usage:
routing_paths = {
    1: [(1, 1, 3), (1, 1, 4), (1, 1, 5), (1, 1, 6)],
    2: [(1, 1, 4), (1, 2, 4), (3, 2, 4), (2, 2, 4), (1, 3, 4)],
    3: []  # Unrouted net
}

output_file_path = '../output/output_test.txt'
generate_output_file(output_file_path, 3, routing_paths)

## 评估函数